# Deploy to Momento Functions

In this final notebook, we will tie together all the pieces we have built in the previous notebooks and deploy our indexer and recommender functions to Momento Functions.

Momento Functions is a serverless compute platform that lets you deploy WebAssembly binaries with zero infrastructure or API Gateway setup. Simply upload your WASM files to Momento, and your code is instantly available at a public HTTP endpoint—no configuration required. You get scalable, production-ready endpoints without managing servers or networking.

For this notebook we will need the following API keys (in the environment variables):
- `MOMENTO_API_KEY`
- `OPENAI_API_KEY`
- `TURBOPUFFER_API_KEY`
- `TURBOPUFFER_NAMESPACE`
- `TURBOPUFFER_REGION`

# Setup

In [ ]:
from momento_buffconf_workshop import NotebookConfiguration

config = NotebookConfiguration.for_functions()
config.print_status_banner()

In [24]:
import base64
import os
from pathlib import Path
from typing import Any, Mapping

from momento import CredentialProvider
import requests

from momento_buffconf_workshop import ArticleContent

Before we add our functions we need to create a namespace in Momento.

In [25]:
# Note that for the workshop you do not need to run this code.
# A cache has already been created for you.

# from datetime import timedelta
# from momento import CacheClient, Configurations

# momento_client = CacheClient(
#     Configurations.Laptop.v1(),
#     CredentialProvider.from_environment_variable("MOMENTO_API_KEY"),
#     default_ttl=timedelta(minutes=5))

# momento_client.create_cache(config.momento_cache_name)


# Minimal client

Momento Functions has a simple HTTP API. In order to reduce boilerplate, we provide a minimal client to abstract away the HTTP details.

In [ ]:
class MomentoFunctionsClient:
    """
    Minimal wrapper for Momento Functions HTTP API.
    Only two calls: put_function() and invoke_function().
    """

    def __init__(self, api_key: str):
        """ Initialize the client with the Momento API key.

        Args:
            api_key (str): Momento API key.

        Raises:
            ValueError: If the API key is not provided.
        """
        if not api_key:
            raise ValueError("Momento API key is required")
        self._api_key = api_key
        self._function_api_prefix = f"https://api.{CredentialProvider.from_string(api_key).cache_endpoint}"

    def put_function(
        self,
        *,
        cache_name: str,
        function_name: str,
        wasm_file_path: Path,
        environment_variables: Mapping[str, str] | None = None,
        timeout: int = 60,
    ) -> dict[str, Any]:
        """Upload or update a Wasm function.

        Args:
            cache_name (str): The name of the cache.
            function_name (str): The name of the function.
            wasm_file_path (Path): The path to the Wasm file.
            environment_variables (Mapping[str, str] | None, optional): Environment variables to set for the function. Defaults to None.
            timeout (int, optional): Timeout for the request in seconds. Defaults to 60.

        Returns:
            dict[str, Any]: The response from the Momento Functions API.
        """
        endpoint = (
            f"{self._function_api_prefix}/functions/manage/{cache_name}/{function_name}"
        )

        with open(wasm_file_path, "rb") as fh:
            inline_wasm_b64 = base64.b64encode(fh.read()).decode()

        payload = {
            "environment_variables": environment_variables or {},
            "inline_wasm": inline_wasm_b64,
        }

        return self._request("PUT", endpoint, json_=payload, timeout=timeout)

    def invoke_function(
        self,
        *,
        cache_name: str,
        function_name: str,
        payload: Any,
        debug: bool = False,
        timeout: int = 60,
    ) -> dict[str, Any]:
        """ Invoke a Momento function.

        Args:
            cache_name (str): The name of the cache where the function is deployed.
            function_name (str): The name of the function to invoke.
            payload (Any): The JSON payload to send to the function.
            debug (bool, optional): Whether to enable debug logging. Defaults to False.
            timeout (int, optional): Timeout for the request in seconds. Defaults to 60.

        Returns:
            dict[str, Any]: The response from the Momento Functions API.
        """
        
        endpoint = (
            f"{self._function_api_prefix}/functions/{cache_name}/{function_name}"
        )

        headers = {}
        if debug:
            headers["x-momento-log"] = "debug"

        return self._request(
            "POST", endpoint, headers=headers, json_=payload, timeout=timeout
        )

    # ---------- internal ------------------------------------------------
    def _request(
        self,
        method: str,
        url: str,
        *,
        json_: Any,
        headers: Mapping[str, str] | None = None,
        timeout: int,
    ) -> dict[str, Any]:
        headers_to_send = {"authorization": self._api_key, "Content-Type": "application/json"}
        if headers:
            headers_to_send.update(headers)

        resp = requests.request(
            method=method,
            url=url,
            headers=headers_to_send,
            json=json_,
            timeout=timeout,
        )
        try:
            resp.raise_for_status()
        except requests.HTTPError as e:
            raise RuntimeError(
                f"Momento Functions API error {resp.status_code}: {resp.text}"
            ) from e

        return resp.json() if resp.text else {}


In [ ]:
client = MomentoFunctionsClient(api_key=os.environ["MOMENTO_API_KEY"])

# Index articles

## Create the indexer function

First let's add the index-articles function to Momento.

In [ ]:
INDEX_ARTICLES_FUNCTION_NAME = "turbopuffer-index-articles"

In [ ]:
client.put_function(
    cache_name=config.momento_cache_name,
    function_name=INDEX_ARTICLES_FUNCTION_NAME,
    wasm_file_path=config.index_articles_function_wasm_path,
    environment_variables={
        "OPENAI_API_KEY": os.environ["OPENAI_API_KEY"],
        "TURBOPUFFER_API_KEY": os.environ["TURBOPUFFER_API_KEY"],
        "TURBOPUFFER_REGION": os.environ["TURBOPUFFER_REGION"],
        "TURBOPUFFER_NAMESPACE": os.environ["TURBOPUFFER_NAMESPACE"],
    },
    timeout=60,
)

## Index our articles

With our production indexer deployed, we can now index our articles.

First let's load our data and inspect it.

In [ ]:
articles = ArticleContent.load_json(config.normalized_article_path)
article_payload = [article.model_dump() for article in articles.all_articles]
article_payload[:5]

Now let's index our articles using the indexer function we just deployed.

In [ ]:
client.invoke_function(
    cache_name=config.momento_cache_name,
    function_name=INDEX_ARTICLES_FUNCTION_NAME,
    payload=article_payload,
    timeout=60,
)

That was easy!

In a production system we would have a cron job to run this periodically, or even triggered on new articles added to our content management system.

# Recommend content

With our indexer live and data indexed, we can now deploy our recommender function to recommend content to our users.

## Create the recommender function

Now put the recommend-articles function in Momento.

In [ ]:
RECOMMEND_ARTICLES_FUNCTION_NAME = "turbopuffer-recommend-articles"

In [ ]:
client.put_function(
    cache_name=config.momento_cache_name,
    function_name=RECOMMEND_ARTICLES_FUNCTION_NAME,
    wasm_file_path=config.recommend_articles_function_wasm_path,
    environment_variables={
        "TURBOPUFFER_API_KEY": os.environ["TURBOPUFFER_API_KEY"],
        "TURBOPUFFER_REGION": os.environ["TURBOPUFFER_REGION"],
        "TURBOPUFFER_NAMESPACE": os.environ["TURBOPUFFER_NAMESPACE"],
    },
    timeout=60,
)

## Recommend content

Recall that our recommender algorithm takes as input the user's content viewing history in order to recommend similar content.

Let's suppose our hypothetical user enjoys Basketball. We can grab some ids from the `NBA` RSS feed we used earlier as a substitute for the user's viewing history.

In [ ]:
nba_articles = articles.articles["nba"][:5]
nba_ids = [article.id for article in nba_articles]
for nba_article in nba_articles:
    print(f"{nba_article.metadata['title']} ==== {nba_article.page_content[:100]}")

That looks good. Let's pass this in as the user's viewing history to our recommender function and see what happens.

In [ ]:
client.invoke_function(
    cache_name=config.momento_cache_name,
    function_name=RECOMMEND_ARTICLES_FUNCTION_NAME,
    payload={
        "article_ids": nba_ids
    },
    timeout=60,
)

Our user would be engaged with the content we recommended!

As a reminder, here are the various RSS feeds we index previously. Try running the flow with various feeds to see how the recommendations change:

In [4]:
list(articles.articles.keys())

['general',
 'boxing',
 'college_basketball',
 'college_football',
 'golf',
 'masters',
 'mlb',
 'mma',
 'nba',
 'nfl',
 'nhl',
 'soccer',
 'tennis',
 'wwe',
 'betting']

# Key takeaways

We deployed our indexer and recommender functions to Momento Functions with almost no boilerplate, no API Gateway to configure, and no infrastructure to manage.